In [ ]:
import numpy as np
from numpy import newaxis
import os

import cv2
import sklearn
from keras.models import Sequential
from keras.layers import Flatten, Dense 
from keras.layers import Lambda, Conv2D, Cropping2D, MaxPool2D


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
path = "data/"
folders = [f for f in os.listdir(path) if os.path.isdir(path+ f)]

In [ ]:
X = []
y = []


print('started!')

for lbl in folders:

    if(lbl == 'Red') :
        y_lbl= 0
    elif(lbl == 'Yellow') :
        y_lbl= 1
    elif(lbl == 'Green'):
        y_lbl= 2
    else:
        y_lbl = 3
    files = [n for n in os.listdir(path+lbl+'/') if (n.endswith('.png')|n.endswith('.jpg'))]
    for n in files:
        img = cv2.cvtColor(cv2.imread(path+lbl+'/'+n), cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256,192))
        X.append(img)
        y.append(y_lbl)
        

y = np.array(y)
X = np.array(X)
X = X/255.0 #normalise! it is vital.
X_, y_ = shuffle(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_, y_, test_size=0.30)

print(y_.shape)
print(X_.shape)

In [ ]:
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Conv2D(24, (5, 5), strides =(2,2), input_shape=(192,256,3), activation="relu"))
model.add(Conv2D(36, (5, 5), strides =(2,2), activation="relu"))
model.add(Conv2D(48, (5, 5), strides =(2,2), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu")) 
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu")) 
model.add(Dense(10, activation="relu"))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size = 128)

In [ ]:
model.save('model_sim-nvidia.h5')